In [1]:
# This is a simple demo for simple gate rotations for one state
from Circuit import Circuit
from Circuit_ops import zero_state, apply, state_prob_plot
from Compiler import compile_gates
from numpy import *
import matplotlib.pyplot as plt
from Gate_bases import *

In [2]:
# Our gate compiler
def compile_gates(gates, errors=[0,0,0]):

    # -- Helper functions for returning gates --
    # The native bit-flip rotation
    def S_phi(q, t, phi):
        return [[s_phi], [q], t, [phi]]

    def X(q, t):
        return S_phi(q, t, 0)

    def Y(q, t):
        return S_phi(q, t, pi/2)

    # Z-rotations are assumed to be synthesized from X and Y for now
    def Z(q, t):
        return Y(q, pi/2), X(q, t), Y(q, -pi/2)

    # XX-rotations are native to ion traps through motional sideband coupling
    def XX(q1, q2, t):
        return [[s_phi, s_phi], [q1, q2], t, [0, 0]]

    native_gates = []
    for gate in gates:

        gate_type = gate[0]     # Gate type
        q = gate[1]             # Qubit(s)
        t = gate[2]             # Gate angle
        phi = gate[3]           # Gate axis (on the x-y plane)

        # -- Switch between different gate types --

        if gate_type == s_phi:
            native_gates.append(S_phi(q, t, phi))

        if gate_type == x:
            native_gates.append(X(q, t))

        if gate_type == y:
            native_gates.append(Y(q, t))

        if gate_type == z:
            native_gates.append(Z(q, t))

        if gate_type == xx:
            native_gates.append(XX(q[0], q[1], t))

        # Hadmard gate synthesis
        if gate_type == h:
            native_gates.extend([Y(q, pi/2), X(q, -pi)])

        # CNOT gate synthesis
        if gate_type == cnot:
            native_gates.extend([Y(q[0], pi/2), XX(q[0], q[1], pi/2),
                                 X(q[0], -pi/2), X(q[1], -pi/2), Y(q[0], -pi/2)])

    compiled_gates = []
    for native_gate in native_gates:
        basis = native_gate[0]
        qubits = native_gate[1]
        
        runs = 2
        # Two-qubit gate
        if len(basis) == 2:
            angle = [native_gate[2] * (xx_err * array(error_dist(runs)) + 1)]
            axis1 = native_gate[3][0] + phase_err * \
                array(error_dist(runs)) * pi * sqrt(2) / 4
            axis2 = native_gate[3][1] + phase_err * \
                array(error_dist(runs)) * pi * sqrt(2) / 4

            compiled_gates.append([basis, qubits, angle, [axis1, axis2]])
            
        # Single-qubit gate
        else:
            angle = [native_gate[2] *
                     (single_err * array(error_dist(runs)) + 1)]
            axis = [native_gate[3][0] +
                    phase_err * array(error_dist(runs)) * pi / 2]

            compiled_gates.append([basis, qubits, angle, axis])

    return native_gates, compiled_gates